[View in Colaboratory](https://colab.research.google.com/github/makumaku/ukiyoe_keras_gan/blob/master/keras_gan_test.ipynb)

In [0]:
# google driveへマウント
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# google drive上の浮世絵のデータセットをcontent配下へフォルダごとコピー
# zipをドライブに入れて、うつそう
!ls -ltr /content
!mkdir /content/ukiyoe_dataset
!cp -r "/content/drive/My Drive/ukiyoe_dataset.zip" /content/
!ls -ltr
#!ls -ltr /content/ukiyoe_dataset
# リサイズフォルダ作成
!mkdir /content/ukiyoe_dataset_resize
# リザルトフォルダ作成
!mkdir result

total 8
drwxr-xr-x 2 root root 4096 Oct 12 16:10 sample_data
drwx------ 2 root root 4096 Oct 14 06:55 drive
total 12400
drwxr-xr-x 2 root root     4096 Oct 12 16:10 sample_data
drwx------ 2 root root     4096 Oct 14 06:55 drive
drwxr-xr-x 2 root root     4096 Oct 14 06:57 ukiyoe_dataset
-rw------- 1 root root 12682076 Oct 14 06:57 ukiyoe_dataset.zip


In [0]:
# 解凍
!unzip /content/ukiyoe_dataset.zip
!ls -ltr /content/ukiyoe_dataset/
#!ls -ltr /content/
#!rm -r /content/ukiyoe_dataset/
#!rm -r /content/ukiyoe_dataset_resize

Archive:  /content/ukiyoe_dataset.zip
  inflating: ukiyoe_dataset/arcBK01-0001_001.jpg  
  inflating: ukiyoe_dataset/arcBK01-0001_009.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_01.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_02.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_03.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_04.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_05.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_06.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_07.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_08.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_09.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_10.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_11.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_12.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_13.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_14.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_15.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_16.jpg  
  inflating: ukiyoe_dataset/arcBK01-0038_17.jpg  
  inflatin

In [0]:
# リサイズ処理
import matplotlib.pyplot as plt
import sys
from keras.preprocessing.image import array_to_img,img_to_array,load_img
import numpy as np
import os
i = os.listdir("/content/ukiyoe_dataset")

from PIL import Image

for f in i:
    img = Image.open("/content/ukiyoe_dataset/"+f)
    img_resize = img.resize((60, 100))
    ftitle, fext = os.path.splitext(f)
    img_resize.save("/content/ukiyoe_dataset_resize/"+ftitle + '_resize' + fext)

Using TensorFlow backend.


In [0]:
X_train=[]

i=os.listdir("/content/ukiyoe_dataset_resize")
n=0
for target_file in i:
  image=("/content/ukiyoe_dataset_resize/"+target_file)
  #print(image)
  temp_img = load_img(image)
  #print(temp_img)
  temp_img_array = img_to_array(temp_img)
  #print(temp_img_array)
  #X_train.append(temp_img_array)
    #X_train.append(image)
    #n=n+1


print(temp_img_array.shape)
X_train = temp_img_array
# X_trainに全ファイルを保存
#np.savez("/content/ukiyoe_dataset_resize/gan.npz",x_train=X_train)
X_train = np.array(X_train)
print(X_train.shape)
#X_train = temp_img

(100, 60, 3)
(100, 60, 3)


In [0]:
# トレーニング用データの名称を変更する。
# 
Y_train = []
Y_train.append(X_train)
Y_train = np.array(Y_train)
#print(Y_train)

In [0]:
## 本番dcgan

from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from keras.backend import tensorflow_backend

import matplotlib.pyplot as plt
import os
import cv2
import numpy as np

np.random.seed(0)
np.random.RandomState(0)
tf.set_random_seed(0)

config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
session = tf.Session(config=config)
tensorflow_backend.set_session(session)

#root_dir = "/home/takusub/PycharmProjects/Samples/dcgan/kill_me_baby_datasets/"


class DCGAN():
    def __init__(self):

        #self.class_names = os.listdir(root_dir)

        self.shape = (100, 60, 3)
        self.z_dim = 100

        optimizer = Adam(lr=0.00002, beta_1=0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        self.generator = self.build_generator()
        # self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        z = Input(shape=(self.z_dim,))
        img = self.generator(z)

        self.discriminator.trainable = False

        valid = self.discriminator(img)

        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):
        noise_shape = (self.z_dim,)
        

        model = Sequential()

        model.add(Dense(128 * 25 * 15, activation="relu", input_shape=noise_shape))
        model.add(Reshape((25, 15, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(3, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        img_shape = self.shape

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def build_combined(self):
        self.discriminator.trainable = False
        ## もし、モデルを使うのであれば、以下を使用。
        model = Sequential([self.generator, self.discriminator])
        #model = keras.models.load_model('/content/dcgan-3300-iter.h5', compile=False)

        return model

    def train(self, iterations, batch_size=128, save_interval=50, model_interval=1000, check_noise=None, r=5, c=5):

        global Y_train
        X_train = Y_train

        half_batch = int(batch_size / 2)

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5

        for iteration in range(iterations):

            # ------------------
            # Training Discriminator
            # -----------------
            idx = np.random.randint(0, X_train.shape[0], half_batch)

            imgs = X_train[idx]

            noise = np.random.uniform(-1, 1, (half_batch, self.z_dim))

            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # -----------------
            # Training Generator
            # -----------------

            noise = np.random.uniform(-1, 1, (batch_size, self.z_dim))

            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))

            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (iteration, d_loss[0], 100 * d_loss[1], g_loss))

            if iteration % save_interval == 0:
                self.save_imgs(iteration, check_noise, r, c)
                start = np.expand_dims(check_noise[0], axis=0)
                end = np.expand_dims(check_noise[1], axis=0)
                resultImage = self.visualizeInterpolation(start=start, end=end)
                cv2.imwrite("/content/result/" + "latent_{}.png".format(iteration), resultImage)
                if iteration % model_interval == 0:
                    self.generator.save("/content/result/dcgan-{}-iter.h5".format(iteration))

    def save_imgs(self, iteration, check_noise, r, c):
        noise = check_noise
        gen_imgs = self.generator.predict(noise)

        # 0-1 rescale
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, :])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig('/content/result/ukiyoe_gen_%d.png' % iteration)

        plt.close()


    def get_class_one_hot(self, class_str):
        label_encoded = self.class_names.index(class_str)

        label_hot = np_utils.to_categorical(label_encoded, len(self.class_names))
        label_hot = label_hot

        return label_hot

    def visualizeInterpolation(self, start, end, save=True, nbSteps=10):
        print("Generating interpolations...")

        steps = nbSteps
        latentStart = start
        latentEnd = end

        startImg = self.generator.predict(latentStart)
        endImg = self.generator.predict(latentEnd)

        vectors = []

        alphaValues = np.linspace(0, 1, steps)
        for alpha in alphaValues:
            vector = latentStart * (1 - alpha) + latentEnd * alpha
            vectors.append(vector)

        vectors = np.array(vectors)

        resultLatent = None
        resultImage = None

        for i, vec in enumerate(vectors):
            gen_img = np.squeeze(self.generator.predict(vec), axis=0)
            gen_img = (0.5 * gen_img + 0.5) * 255
            interpolatedImage = cv2.cvtColor(gen_img, cv2.COLOR_RGB2BGR)
            interpolatedImage = interpolatedImage.astype(np.uint8)
            resultImage = interpolatedImage if resultImage is None else np.hstack([resultImage, interpolatedImage])

        return resultImage


if __name__ == '__main__':
    dcgan = DCGAN()
    r, c = 5, 5
    check_noise = np.random.uniform(-1, 1, (r * c, 100))
    dcgan.train(iterations=200000, batch_size=32, save_interval=300, model_interval=300, check_noise=check_noise, r=r,
                c=c)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 50, 30, 32)        896       
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 50, 30, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 50, 30, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 25, 15, 64)        18496     
_________________________________________________________________
zero_padding2d_4 (ZeroPaddin (None, 26, 16, 64)        0         
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 26, 16, 64)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 26, 16, 64)        0         
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.003897, acc.: 31.25%] [G loss: 1.059119]
Generating interpolations...
1 [D loss: 0.804637, acc.: 50.00%] [G loss: 0.831656]
2 [D loss: 0.812854, acc.: 46.88%] [G loss: 1.096459]
3 [D loss: 0.613886, acc.: 68.75%] [G loss: 1.023986]
4 [D loss: 0.481100, acc.: 78.12%] [G loss: 1.120036]
5 [D loss: 0.571785, acc.: 71.88%] [G loss: 0.952089]
6 [D loss: 0.401518, acc.: 90.62%] [G loss: 1.101301]
7 [D loss: 0.438835, acc.: 84.38%] [G loss: 1.060569]
8 [D loss: 0.474845, acc.: 87.50%] [G loss: 0.944772]
9 [D loss: 0.364875, acc.: 81.25%] [G loss: 1.090271]
10 [D loss: 0.363258, acc.: 93.75%] [G loss: 1.190432]
11 [D loss: 0.357599, acc.: 90.62%] [G loss: 1.216716]
12 [D loss: 0.342597, acc.: 93.75%] [G loss: 1.328068]
13 [D loss: 0.320870, acc.: 90.62%] [G loss: 1.221325]
14 [D loss: 0.261681, acc.: 93.75%] [G loss: 1.343540]
15 [D loss: 0.262808, acc.: 100.00%] [G loss: 1.358359]
16 [D loss: 0.316946, acc.: 84.38%] [G loss: 1.316536]
17 [D loss: 0.232578, acc.: 100.00%] [G loss:

KeyboardInterrupt: ignored

In [0]:
!mkdir "/content/drive/My Drive/20181011"
#!ls -ltr /content/result
!cp /content/result/*.* "/content/drive/My Drive/20181011/"
#!cp "/content/drive/My Drive/dcgan-3300-iter.h5" /content/
#!ls -ltr

In [0]:
!cp "/content/drive/My Drive/cnn_weight.h5" /content/
!cp "/content/drive/My Drive/cnn_model.h5" /content/